# Portfolio selection to maximize returns
*Problem Statement* : Given asset price history and a budget (say $100,000),  determine maximum possible returns for a given past period.
*Strategy* : On each day of the investment period, invest the entire budget on the asset which gave the highest return for that day.

**Data Preparation**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from datetime import datetime as dt
from scipy.optimize import linprog
from kaggle.competitions import twosigmanews 

import os
print(os.listdir("../input"))
env = twosigmanews.make_env()
print(os.listdir("../input"))
(market_df, new_df) = env.get_training_data()

# Convert the data type of 'time' column from datetime to date
market_df['time']=  market_df['time'].dt.date

# Copy the required columns
market_df_sorted = market_df[['time','assetCode','close','open','returnsOpenPrevRaw1']].copy(deep=True)

# Sort the dataframe by date. Then reset the dataframe indices.
market_df_sorted = market_df.sort_values(by=['time'])
market_df_sorted.index = pd.RangeIndex(len(market_df_sorted.index))

# Index and it's max value, to iterate through the market data dataframe
dfIndex = 0
dfSize = len(market_df_sorted)

assetAvgList = []
assetCount = 0

**Setting the Test Period **

In [ ]:
# Input the start and end dates for the time period to be considered

print ("\n Enter test period start date (eg: 20 Dec 2014) :")
periodStart = input()
print("\n Enter test period end date (eg: 5 Jun 2014) :")
periodEnd = input()

# Convert the input string values to datetime objects and then to date objects
periodStart = dt.strptime(periodStart,'%d %b %Y')
periodStart = periodStart.date()
periodEnd = dt.strptime(periodEnd,'%d %b %Y')
periodEnd = periodEnd.date()

**Picking a random sample of assets from the dataset**

In [ ]:
# Pick a sample of a few assets from the data and store it in a dictionary of dataframes - each asset in a dataframe

listIndex = 0        # Index to interate thru the list of sampled asset codes
sampleSize = 10      # Number of assets to be picked for this program

uniqueAssets = market_df_sorted.assetCode.unique()            # List of all unique assets in the data
numOfAssets = len(uniqueAssets)     
print("\n Total number of unique assets in the dataset =",numOfAssets)

assetSampleList = np.random.choice(uniqueAssets,sampleSize)   # Pick a random sample of assets to test

# Create a summary dataframe to store the asset codes and their number of occurances
testAssetsSummary = pd.DataFrame(columns=['assetCode','count'])

# Initiate the summary dataframe with the asset codes, and count value as 0
while (listIndex < sampleSize) :    
    testAssetsSummary=testAssetsSummary.append({'assetCode':assetSampleList[listIndex], 'count':0},ignore_index=True)
    listIndex += 1
print(testAssetsSummary)


**Creating data collections **
- The data doesn't include all dates - only those dates on which trading was open. Pick out those dates to enable calculations.
- Group the selected data by asset codes, and by trading dates.

In [ ]:
testDateList = list()
type(testDateList)
dfIndex=0
listIndex = 0

# Create 2 collections to hold all data from the sampled assets, for the test period

# 1 - a dictionary of dataframes with key = assetCode, here the data is grouped by asset
testDfDict_A = dict.fromkeys(assetSampleList,pd.DataFrame())

# Loop to copy data to the test collections, and calculate 'count' for each asset
while dfIndex<dfSize :
    currDate = market_df_sorted.at[dfIndex,'time'] 
    if currDate<periodStart :
        dfIndex+=1
        continue
    elif currDate>periodEnd :
        dfIndex+=1
        break

# Create a list of the trading days (dates with data) in the selected time period    
    if currDate != market_df_sorted.at[dfIndex+1,'time'] :
        testDateList.insert(listIndex, currDate)
        testDfDict_D[currDate]=pd.DataFrame()
        listIndex+=1

# Copy only the data from the selected assets and the required test period
    currAsset = market_df_sorted.at[dfIndex,'assetCode']
    if currAsset in assetSampleList :
        tempRowData = market_df_sorted.loc[dfIndex]
        testDfDict_A[currAsset]=testDfDict_A[currAsset].append(tempRowData,ignore_index=True)

# Increment 'count' - the number of occurances counter for each asset    
    testAssetsSummary.loc[testAssetsSummary.assetCode==currAsset,'count']+=1
    
    dfIndex+=1
print("\n List of all trading dates from the selected period :\n\n")
testDateList

In [ ]:
# 2 - a dictionary of dataframes with key = time, here the data is grouped by date
testDfDict_D = dict.fromkeys(testDateList,pd.DataFrame())
dfIndex=0
while dfIndex<dfSize :
    currDate=market_df_sorted.at[dfIndex,'time']
    currAsset=market_df_sorted.at[dfIndex,'assetCode']
    if currDate<periodStart :
        dfIndex+=1
        continue
    elif currDate>periodEnd :
        dfIndex+=1
        break
    if currAsset in assetSampleList :
        tempRowData=market_df_sorted.loc[dfIndex]
        testDfDict_D[currDate]=testDfDict_D[currDate].append(tempRowData,ignore_index=True)
    dfIndex+=1
testDfDict_D

**Comparing returns and picking the portfolio**
- Find which asset gives the maximum return on each day.
- Store the asset code corresponding to the maximum return in the portfolio, for that date

In [19]:
# 'portfolio' dataframe to hold details of which asset to invest in each day
portfolio = pd.DataFrame(columns=['date','assigned','assetCode','returns'])

# Calculate the highest possible return on each day
maxDailyReturns = dict.fromkeys(testDateList)
dictIndex = 0
maxRet=0
dateSize=len(testDateList)

while (dictIndex<dateSize) :
    currDate = testDateList[dictIndex]
    maxDailyReturns[currDate] = testDfDict_D[currDate]['returnsOpenPrevRaw1'].max()
    currAsset = testDfDict_D[currDate].loc[testDfDict_D[currDate]['returnsOpenPrevRaw1']==maxDailyReturns[currDate],'assetCode'].item()
    if maxDailyReturns[currDate]>0 :
        row = {'date':currDate,'assigned':True,'assetCode':currAsset,'returns':maxDailyReturns[currDate]}
        portfolio=portfolio.append(row,ignore_index=True)
    else :
        row = {'date':currDate,'assigned':False,'assetCode':'None','returns':0}
        portfolio=portfolio.append(row,ignore_index=True)
    dictIndex += 1
print(portfolio)

          date assigned assetCode   returns
0   2009-05-01     True     KMX.N  0.008703
1   2009-05-04     True     DLM.N  0.029333
2   2009-05-05     True     DLM.N  0.033679
3   2009-05-06     True     KMX.N  0.080997
4   2009-05-07     True     TIa.N  0.028666
5   2009-05-08     True     KTC.N  0.008208
6   2009-05-11     True     NHP.N  0.013555
7   2009-05-12     True     TIa.N  0.048087
8   2009-05-13     True     KTC.N  0.004639
9   2009-05-14     True     DLM.N  0.010063
10  2009-05-15     True      MA.N  0.016235
11  2009-05-18     True     TIa.N  0.040349
12  2009-05-19     True     NHP.N  0.042783
13  2009-05-20     True     TIa.N  0.021830
14  2009-05-21    False      None  0.000000
15  2009-05-22     True     NHP.N  0.018371
16  2009-05-26    False      None  0.000000
17  2009-05-27     True     KMX.N  0.073695
18  2009-05-28     True     KTC.N  0.036870
19  2009-05-29     True     KMX.N  0.041667
20  2009-06-01     True     KTC.N  0.047861


On certain days, all the sampled assets could have negative returns. Since the goal is to maximize returns, no ivestment should have been made on such days. Thus, the resultant portfolio could have 'None' as the asset to be invested in, for some days in the test period.

In [17]:
print("\n Portfolio for maximum return :\n\n ",portfolio[['date','assetCode']])
#print("\n Maximized return = $"+"{0:.3f}".format(100000*portfolio.returns.sum()))


 Portfolio for maximum return :

            date assetCode
0   2009-05-01     KMX.N
1   2009-05-04     DLM.N
2   2009-05-05     DLM.N
3   2009-05-06     KMX.N
4   2009-05-07     TIa.N
5   2009-05-08     KTC.N
6   2009-05-11     NHP.N
7   2009-05-12     TIa.N
8   2009-05-13     KTC.N
9   2009-05-14     DLM.N
10  2009-05-15      MA.N
11  2009-05-18     TIa.N
12  2009-05-19     NHP.N
13  2009-05-20     TIa.N
14  2009-05-21      None
15  2009-05-22     NHP.N
16  2009-05-26      None
17  2009-05-27     KMX.N
18  2009-05-28     KTC.N
19  2009-05-29     KMX.N
20  2009-06-01     KTC.N

 Maximized return = $60558.943


# Portfolio selection using predicted asset values
*Objective*: Use historic asset prices to forecast future prices and pick portfolio which would maximize returns on the next day.
*Strategy*: Use time-series forecasting methods to predict asset prices and determine the optimum portfolio by using a linear programming model.

In [15]:
#SIMPLE AVG RAW RETURNS
iterIndex=0
assetsPresent = testDfDict_D[testDateList[0]].assetCode.unique()
# Create a dictionary of size 30 to store the average returns of the sampled assets
avgRetSA = dict.fromkeys(assetSampleList,0)

# Loop to calculate average returns for each asset
for iterName in avgRetSA.keys() :
    # Condition to handle sampled assets which have no data in the given time period
    if iterName in assetsPresent :
        tempFrame = testDfDict_A[iterName]
        avgRetSA[iterName] = tempFrame['returnsOpenPrevRaw1'].sum()/len(tempFrame.index)
    
print(avgRetSA)
print("\nMaximum avg return for an asset =",max(avgRetSA.values()),"%)")

# If the asset occurs only once in the dataframe, while condition will fail and
# assetReturnsSum will remain 0. The following if clause remedies this case.
    

{'PMI.N': 0, 'DLM.N': 0.004049915807019178, 'TRID.O': 0, 'TIa.N': 0.005070854713950836, 'SAH.N': 0, 'KTC.N': 0.0002262776152329462, 'MA.N': -0.0006920117549769278, 'NHP.N': 0.003918602531340217, 'KMX.N': -0.0041994484519927685, 'USCR.O': 0}

Maximum avg return for an asset = 0.005070854713950836 %)


In [16]:
#MAXIMIZE RETURNS USING SA FORECAST

#x_Opt = np.zeros(sampleSize, dtype=float)
c = list(avgRetSA.values())
neg = -1
cNeg = neg*np.array(c)

A_eq = np.array([[1]*sampleSize])
b_eq = np.array([100000])

res=linprog(cNeg,A_eq=A_eq,b_eq=b_eq,bounds=(0,100000))

print("Maximized Return for the next day = $"+"{0:.3f}".format(res.fun*-1))
print("\n Amount to be invested in each asset :")
printList = list(avgRetSA.keys())
for i in range(sampleSize) :
    print("\n",printList[i]," = $"+"{0:.3f}".format(res.x[i]))

Maximized Return = $507.085

 Amount to be invested in each asset :

 PMI.N  = $0.000

 DLM.N  = $0.000

 TRID.O  = $0.000

 TIa.N  = $100000.000

 SAH.N  = $0.000

 KTC.N  = $0.000

 MA.N  = $0.000

 NHP.N  = $0.000

 KMX.N  = $0.000

 USCR.O  = $0.000
